<a href="https://colab.research.google.com/github/RoshanTS/extract-2/blob/master/extraction_donn%C3%A9es_via_api_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On va extraire les tweets 

sdfdsfdsfdsfdssdfdsf
